In [290]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime


In [291]:
# Get population data
pop_file_name = os.path.join("..","..","population_files_Feb8","population_file_sent_Feb8.xlsx")
pop_df = pd.read_excel(pop_file_name)

# Get logistic regression betas
beta2345_f = os.path.join("..","..","Output_SM","Betas","Beta_Estimates_2345.xlsx")
beta1_f = os.path.join("..","..","Output_SM","Betas","Beta_Estimates_1.xlsx")
beta2345 = pd.read_excel(beta2345_f).to_numpy()[:,2:].astype(np.float64)
beta1 = pd.read_excel(beta1_f).to_numpy()[:,2:].astype(np.float64)
print(beta1)

[[ 5.98310348e+00 -2.02291154e-01  6.51019669e-04  6.57941237e-01
  -8.33318673e-01]
 [ 1.79539529e+00 -2.70409828e-01 -8.19793058e-03  1.34197520e-01
  -2.60933105e-01]
 [ 5.01740931e-01  1.44754871e+00 -6.96110007e-02  8.97391797e-01
   4.68685197e-01]
 [ 5.56110810e+00 -2.83234113e-01 -2.01464620e-01 -2.41575963e-01
  -9.73101597e-01]]


In [292]:
# pop_arr = pop_df.to_numpy(dtype=np.float64)

# arr2345 = np.asarray([row for row in pop_arr 
#         if (row[4] == 2 or row[4] == 3 or row[4] == 4 or row[4] == 5
#         or row[3] == 2 or row[3] == 3 or row[3] == 4 or row[3] == 5)], dtype=np.float64)
# arr1 = np.asarray([row for row in pop_arr 
#         if (row[4] == 1 and row[3] == 1)], dtype=np.float64)
# arr6 = None

In [293]:
def path_to_indicator_form(a):
    s2 = a[:,3]
    s3 = a[:,4]
    ia = a[:,5]
    a = np.concatenate([
        np.ones((len(a), 1)),
        (s2 == 1)[:,np.newaxis],
        (s2 == 2)[:,np.newaxis],
        (s2 == 3)[:,np.newaxis],
        (s2 == 4)[:,np.newaxis],
        (s3 == 2)[:,np.newaxis],
        (s3 == 3)[:,np.newaxis],
        (s3 == 4)[:,np.newaxis],
        (ia == 1)[:,np.newaxis],
        (ia == 2)[:,np.newaxis],
        a[:,2][:,np.newaxis], # black
        a[:,9][:,np.newaxis], # age
        a[:,1][:,np.newaxis] - 1, # change sex from {1,2} to {0,1}
        a[:,6][:,np.newaxis],  # poverty is already {0,1} now, not {1,2} like before
        a[:,10][:,np.newaxis], # start age
        a[:,8][:,np.newaxis], # weight
        -1 * np.ones((a.shape[0],1)), # year last smoked initialize to -1 for nonsmokers
    ], axis=1, dtype=np.float64)
    return a

# arr2345 = path_to_indicator_form(arr2345)
# arr1 = path_to_indicator_form(arr1)

# # TODO: put this stuff in the path_to_indicator_form function
# # Here we figure out the year_last_smoked variable for all cases

# # for people whose last state is 3,4 the year last smoked is self.start_year - 1
# arr2345[np.logical_or(arr2345[:,3],arr2345[:,4]),16] = self.start_year - 1

# # for people currently in groups 3,4 the year last smoked is self.start_year
# arr2345[np.logical_or(arr2345[:,6],arr2345[:,7]),16] = self.start_year

# # for people whose last state is 5, the year last smoked is self.start_year - 1
# # we are treating ecig users the same as smokers here
# arr2345[np.sum(arr2345[:,1:5], axis=1) == 0,16] = self.start_year

# # for people whose current state is 5, the year last smoked is self.start_year
# arr2345[np.sum(arr2345[:,5:8], axis=1) == 0,16] = self.start_year

# # for people in group 2 last state AND this state
# # if initialization age is 1 then year last smoked is self.year_last_smoked_for_ia1 + self.start_year - age
# ind = np.logical_and(arr2345[:,2], arr2345[:,5], arr2345[:,8]).astype(np.bool_)
# arr2345[ind,16] = self.age_last_smoked_for_ia1 + self.start_year - arr2345[ind, 11]

# # if initialization age is 2 for former smokers then year last smoked is randomly chosen between start_age and current age
# ind = np.logical_and(arr2345[:,2], arr2345[:,5], arr2345[:,9]).astype(np.bool_)
# age_started = np.maximum(18, arr2345[ind,14]) # use starting age if available, otherwise use 18
# to_multiply_rand = arr2345[ind, 11] - age_started + 1 - 1e-8
# to_add_after_multiply = self.start_year - arr2345[ind, 11] - 0.5 + 1e-8
# arr2345[ind ,16] = np.round(np.random.rand(np.sum(ind)) * to_multiply_rand + to_add_after_multiply)
                    
# # now the population arrays are in the right format for matrix mult

In [294]:
l = []

# 0 agegrp
# 1 sex
# 2 black
# 3 state2
# 4 state3
# 5 ia
# 6 pov
# 7 set
# 8 weight
# 9 age
# 10 start_age

for s2 in range(1,6):
    for s3 in range(1,6):
        for black in range(2):
            for sex in range(1,3):
                for pov in range(2):
                    for ia in range(3):
                        l.append([
                            0,
                            sex,
                            black, 
                            s2, 
                            s3, 
                            ia,
                            pov, 
                            0, 
                            0, 
                            30, 
                            20, 
                        ])

In [295]:
len(l)

600

In [296]:
l[0]

[0, 1, 0, 1, 1, 0, 0, 0, 0, 30, 20]

In [297]:
pop_arr = np.asarray(l)

arr2345_old = np.asarray([row for row in pop_arr 
            if (row[4] == 2 or row[4] == 3 or row[4] == 4 or row[4] == 5
            or row[3] == 2 or row[3] == 3 or row[3] == 4 or row[3] == 5)], dtype=np.float64)
arr1_old = np.asarray([row for row in pop_arr 
            if (row[4] == 1 and row[3] == 1)], dtype=np.float64)
arr6 = None

In [298]:
arr2345 = path_to_indicator_form(arr2345_old)
arr1 = path_to_indicator_form(arr1_old)


In [299]:
beta_2345_aug = np.concatenate([
    beta2345,
    np.zeros((len(beta2345), 3), dtype=np.float64),
], axis=1, dtype=np.float64)

beta_1_aug = np.concatenate([
    beta1[:,0][:,np.newaxis],
    np.zeros((len(beta1), 9), dtype=np.float64),
    beta1[:,1:],
    np.zeros((len(beta1), 3), dtype=np.float64),
], axis=1, dtype=np.float64)

beta_2345_aug = np.transpose(beta_2345_aug)
beta_1_aug = np.transpose(beta_1_aug)

In [300]:
logits_2345 = np.matmul(arr2345, beta_2345_aug).astype(np.float64)
assert(logits_2345.shape[1] == 3)

logits_1 = np.matmul(arr1, beta_1_aug).astype(np.float64)
assert(logits_1.shape[1] == 4)

# convert logits to probabilities

exps = np.exp(logits_2345)
p4 = 1 / (1 + np.sum(exps, axis=1))
probs2345 = np.asarray([
    np.zeros(arr2345.shape[0]), # p1
    p4*exps[:,0], # p2
    p4*exps[:,1], # p3
    p4,           # p4
    p4*exps[:,2], # p5
], dtype=np.float64).transpose()

exps = np.exp(logits_1)
p4 = 1 / (1 + np.sum(exps, axis=1))
probs1 = np.asarray([
    p4*exps[:,0], # p1
    p4*exps[:,1], # p2
    p4*exps[:,2], # p3
    p4,           # p4
    p4*exps[:,3], # p5
], dtype=np.float64).transpose()


In [301]:
probs2345.shape

(576, 5)

In [302]:
together_pop_probs2345 = np.concatenate([arr2345_old, probs2345], axis=1)
together_pop_probs1 = np.concatenate([arr1_old, probs1], axis=1)

In [303]:
l = list(pop_df.columns)
l.append("prob 1")
l.append("prob 2")
l.append("prob 3")
l.append("prob 4")
l.append("prob 5")
l

['agegrp',
 'R01R_A_SEX_IMP',
 'black',
 'state_2',
 'state_3',
 'initiation_age_grp',
 'poverty1',
 'set',
 'weight',
 'age',
 'start_age',
 'prob 1',
 'prob 2',
 'prob 3',
 'prob 4',
 'prob 5']

In [304]:
together_df2345 = pd.DataFrame(data=together_pop_probs2345, index=None, columns=l)
together_df1 = pd.DataFrame(data=together_pop_probs1, index=None, columns=l).drop(columns=["initiation_age_grp", "agegrp", "set", "weight", "start_age"]).drop_duplicates().reset_index().drop(columns=["index"]).drop_duplicates().reset_index().drop(columns=["index"])

In [305]:
together_df2345.loc[194]

agegrp                 0.000000
R01R_A_SEX_IMP         1.000000
black                  0.000000
state_2                2.000000
state_3                5.000000
initiation_age_grp     2.000000
poverty1               0.000000
set                    0.000000
weight                 0.000000
age                   30.000000
start_age             20.000000
prob 1                 0.000000
prob 2                 0.482867
prob 3                 0.064060
prob 4                 0.128239
prob 5                 0.324833
Name: 194, dtype: float64

In [306]:
together_df1

,R01R_A_SEX_IMP,black,state_2,state_3,poverty1,age,prob 1,prob 2,prob 3,prob 4,prob 5
0,1.0,0.0,1.0,1.0,0.0,30.0,0.984111,0.011457,0.000498,0.002433,0.001501
1,1.0,0.0,1.0,1.0,1.0,30.0,0.971337,0.020043,0.001807,0.005525,0.001288
2,1.0,0.0,1.0,1.0,1.0,30.0,0.971337,0.020043,0.001807,0.005525,0.001288
3,2.0,0.0,1.0,1.0,0.0,30.0,0.990649,0.006831,0.000637,0.001268,0.000615
4,2.0,0.0,1.0,1.0,0.0,30.0,0.990649,0.006831,0.000637,0.001268,0.000615
5,2.0,0.0,1.0,1.0,1.0,30.0,0.982250,0.012005,0.002321,0.002894,0.000530
6,2.0,0.0,1.0,1.0,1.0,30.0,0.982250,0.012005,0.002321,0.002894,0.000530
7,1.0,1.0,1.0,1.0,0.0,30.0,0.982374,0.010683,0.002587,0.002973,0.001382
8,1.0,1.0,1.0,1.0,0.0,30.0,0.982374,0.010683,0.002587,0.002973,0.001382
9,1.0,1.0,1.0,1.0,1.0,30.0,0.964184,0.018585,0.009336,0.006714,0.001180


In [307]:
together_df2345.to_excel("test_outputs_2345.xlsx")
together_df1.to_excel("test_outputs_1.xlsx")